In [ ]:
!pip install kagglehub
import kagglehub
kagglehub.login()

In [ ]:
import os
from google.colab import userdata
os.environ["username"] = userdata.get('username')
os.environ["key"] = userdata.get('key')

In [ ]:
!pip install -q -U keras-nlp
!pip install -U keras

In [ ]:
os.environ["KERAS_BACKEND"] = "jax"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [ ]:
import pandas as pd


file_path = 'combined.csv'
df = pd.read_csv(file_path)


print(df.head())

In [ ]:
instruction_col = 'prompt'
response_col = 'response'

data = []
template = "Instruction:\n{instruction}\n\nResponse:\n{response}"

for _, row in df.iterrows():
    data.append(template.format(instruction=row[instruction_col], response=row[response_col]))


data = data[:1000]

In [ ]:
data[10]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import keras
import keras_nlp

# Load the GemmaCausalLM model
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

# Enable LoRA on the model
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

# Set the sequence length for the preprocessor
gemma_lm.preprocessor.sequence_length = 512

# Initialize the optimizer with AdamW
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)

# Exclude certain variables from weight decay
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

# Compile the model with loss and metrics
gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Assuming `data` is a preprocessed dataset
# Fit the model to the data
gemma_lm.fit(data, epochs=1, batch_size=1)

# Save the model to Google Drive in native Keras format
model_path = '/content/drive/MyDrive/Gemma/gemma_lm_model.keras'
gemma_lm.save(model_path)


In [ ]:
import keras
import keras_nlp


gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()


gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()


gemma_lm.preprocessor.sequence_length = 512


optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)


optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])


gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)


gemma_lm.fit(data, epochs=1, batch_size=1)


In [ ]:
prompt = template.format(
    context
    instruction="",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

In [ ]:
model_path = 'gemma_lm_model.keras'
gemma_lm.save(model_path)

In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras_nlp
import tensorflow_text as text


model_path = '/content/drive/MyDrive/Gemma/gemma_lm_model.keras'

custom_objects = {
    'GemmaCausalLM': keras_nlp.models.GemmaCausalLM,
    'GemmaCausalLMPreprocessor': keras_nlp.models.GemmaCausalLMPreprocessor,
    'GemmaTokenizer': keras_nlp.models.GemmaTokenizer,
}

gemma_lm = keras.models.load_model(model_path, custom_objects=custom_objects)

In [ ]:
import gradio as gr
import tensorflow as tf
from tensorflow import keras
import keras_nlp
import tensorflow_text as text


model_path = '/content/drive/MyDrive/Gemma/gemma_lm_model.keras'

custom_objects = {
  'GemmaCausalLM': keras_nlp.models.GemmaCausalLM,
  'GemmaCausalLMPreprocessor': keras_nlp.models.GemmaCausalLMPreprocessor,
  'GemmaTokenizer': keras_nlp.models.GemmaTokenizer,
}

gemma_lm = keras.models.load_model(model_path, custom_objects=custom_objects)

# Function to generate responses
def generate_response(prompt):
    formatted_prompt = f"{prompt}\nResponse:"
    try:
        response = gemma_lm.generate(formatted_prompt, max_length=256)
        return response
    except Exception as e:
        return str(e)

# Create Gradio interface
interface = gr.Interface(
    fn=generate_response,
    inputs=gr.Textbox(lines=2, placeholder="Enter your instruction here..."),
    outputs=gr.Textbox(lines=10, placeholder="Generated response will appear here..."),
    title="Gemma Language Model",
    description="Generate responses using the Gemma Language Model"
)

# Launch the Gradio app
interface.launch()


In [ ]:
!pip install keras-nlp


In [ ]:
!pip install gradio